# Upgrad Grid World
## MC 방법론 사용
#### 한 에피소드의 경험을 쌓고
#### 경험한 데이터로 q(s,a) 테이블의 값을 업데이트하고 (정책 평가)
#### 업데이트된 q(s,a) 테이블을 이용하여 e-greedy 정책을 만들고 (정책 개선)

In [1]:
import random
import numpy as np

In [5]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0

    def step(self, a):
        #Action 0 : left, 1 : up, 2 : right, 3 : down
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()
        
        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_left(self):
        if self.y == 0: #벽에 막히는 경우는 무시하기 위함
            pass
        elif self.y == 3 and self.x in [0,1,2]:
            pass
        elif self.y == 5  and self.x in [2,3,4]:
            pass
        else:
            self.y -= 1
        
    def move_right(self):
        if self.y == 1 and self.x in [0,1,2]:
            pass
        elif self.y == 3 and self.x in [2,3,4]:
            pass
        elif self.y == 6:
            pass
        else:
            self.y += 1
        
    def move_up(self):
        if self.x == 0:
            pass
        elif self.x == 3 and self.y == 2:
            pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x == 4:
            pass
        elif self.x == 1 and self.y == 4:
            pass
        else:
            self.x += 1
        
    def is_done(self):
        if self.x == 4 and self.y == 6: #목표 지점인 (4,6)에 도달하면 끝
            return True
        else:
            return False
        
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [7]:
#Q-Agent Class
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5,7,4)) #Q value를 저장하기 위한 변수 (모두 0으로 초기화)
        self.eps = 0.9 #e-greedy 값
        self.alpha = 0.01 

    def select_action(self, s):
        x, y = s #state
        coin = random.random()
        if coin < self.eps : #coin 값이 eps보다 낮을 경우 Exploration 진행 (random action)
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action
    
    def update_table(self, history):
        cum_reward = 0
        for transition in history[::-1]:
            s, a, r, s_prime = transition
            x, y = s
            #MC 방법론을 이용한 업데이트
            self.q_table[x,y,a] = self.q_table[x,y,a] + self.alpha * (cum_reward - self.q_table[x,y,a])
            cum_reward = cum_reward + r

    def  anneal_eps(self): #decayin e-greedy를 위해 선형적으로 수치 감소
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self): #학습이 각 위치에서 어느 Action의 Q Value가 가장 높았는지 보여주는 함수
        q_lst = self.q_table.tolist()
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [18]:
#main 함수
def main():
    env = GridWorld()
    agent = QAgent()

    for n_epi in range(1000):
        done = False
        history = []

        s = env.reset()
        while not done: #한 Episode가 끝날 때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime
        agent.update_table(history)
        agent.anneal_eps()

    agent.show_table()

In [28]:
main()

[[3. 2. 0. 2. 2. 3. 0.]
 [3. 3. 0. 1. 1. 2. 3.]
 [3. 2. 0. 1. 0. 3. 3.]
 [2. 3. 2. 1. 0. 3. 3.]
 [2. 2. 1. 1. 0. 2. 0.]]


# SARSA - TD Control
## S(State) ▶ A(Action) ▶ R(Reward) ▶ S'(State') ▶ A'(Action')
### TD는 분산이 작다는 MC 보다 큰 장점을 가지고 있는데 이를 활용하기 위해서는 SARSA에 대해서 이해를 먼저 해야한다.
### 기댓값 안에 Sample을 무수히 모은 후 정책 함수를 이용해 자유롭게 나아가며 각 step 늘어날 때 그 데이터를 이용해 TD 타깃을 계산


In [23]:
#SARSA 구현
class SARSA():
    def __init__(self):
        self.q_table = np.zeros((5,7,4))
        self.eps = 0.9

    def select_action(self, s):
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) #s'에서 선택할 action(실제로 취한 action이 아님)
        #SARSA 업데이트 식을 이용
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1 * (r + self.q_table[next_x, next_y, a_prime]-self.q_table[x,y,a])

    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)
    
    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeros((5,7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [29]:
#SARSA main
def main_sarsa():
    env = GridWorld()
    agent = SARSA()

    for n_epi in range(1000):
        done = False

        s = env.reset()
        while not done:
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s,a,r,s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()

In [30]:
main_sarsa()

[[3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [3. 2. 2. 1. 0. 3. 3.]
 [0. 2. 3. 0. 0. 2. 0.]]
